In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load and preprocess data
df = pd.read_csv("indian_food.csv")

def clean_ingredients(text):
    if pd.isnull(text):
        return ""
    return " ".join(str(text).lower().replace(",", " ").split())

df["ingredients_clean"] = df["ingredients"].apply(clean_ingredients)
df = df[df["ingredients_clean"].str.strip() != ""]

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["ingredients_clean"])

def tfidf_recipe_chatbot(user_input, diet_preference):
    user_input_clean = clean_ingredients(user_input)
    if not user_input_clean:
        return "⚠️ Please enter some ingredients to get recipe suggestions."

    # Handle diet preference
    diet_preference = diet_preference.lower()
    if diet_preference in ['veg', 'vegetarian']:
        mask = df['diet'] == 'vegetarian'
    elif diet_preference in ['non-veg', 'non vegetarian', 'nonveg']:
        mask = df['diet'] == 'non vegetarian'
    else:
        mask = pd.Series(True, index=df.index)

    valid_indices = np.where(mask)[0]
    if not valid_indices.size:
        return "❌ No recipes found for the specified diet."

    # Transform input and calculate similarities
    user_tfidf = vectorizer.transform([user_input_clean])
    tfidf_subset = tfidf_matrix[valid_indices]
    similarities = cosine_similarity(user_tfidf, tfidf_subset).flatten()

    # Get top matches
    top_indices = similarities.argsort()[::-1][:5]
    top_indices = [i for i in top_indices if similarities[i] > 0]

    if not top_indices or similarities[top_indices[0]] < 0.2:
        return "❌ No matches found. Try different ingredients or diet."

    # Prepare response
    response = "🔍 Top matching recipes:\n"
    for idx in top_indices:
        original_idx = valid_indices[idx]
        recipe = df.iloc[original_idx]
        response += (
            f"\n🍽️ {recipe['name']} ({recipe['diet'].title()})\n"
            f"Ingredients: {recipe['ingredients']}\n"
            f"Match Score: {similarities[idx]:.2f}\n"
        )

    return response

# Chat interface
print("👩‍🍳 Smart Recipe Chatbot (Veg/Non-Veg Filter)")
while True:
    ingredients = input("\nEnter ingredients (comma-separated): ").strip()
    if ingredients.lower() == 'exit':
        print("👋 Happy cooking!")
        break

    diet = input("Diet preference (veg/non-veg/any): ").strip()
    print(tfidf_recipe_chatbot(ingredients, diet))


👩‍🍳 Smart Recipe Chatbot (Veg/Non-Veg Filter)
🔍 Top matching recipes:

🍽️ Chhena poda (Vegetarian)
Ingredients: Sugar, chenna cheese
Match Score: 0.62

🍽️ Kalakand (Vegetarian)
Ingredients: Milk, cottage cheese, sugar
Match Score: 0.60

🍽️ Shahi paneer (Vegetarian)
Ingredients: Cottage cheese, malai, garam masala, ginger, tomato
Match Score: 0.44

🍽️ Palak paneer (Vegetarian)
Ingredients: Cottage cheese, palak, cream, garam masala, butter
Match Score: 0.42

🍽️ Kadai paneer (Vegetarian)
Ingredients: Cottage cheese, bell peppers, gravy, garam masala, cashew nuts
Match Score: 0.33

🔍 Top matching recipes:

🍽️ Bebinca (Vegetarian)
Ingredients: Coconut milk, egg yolks, clarified butter, all purpose flour
Match Score: 0.43

🍽️ Chicken Tikka (Non Vegetarian)
Ingredients: Chicken, whole wheat bread, rice flour, garam masala powder, whole egg
Match Score: 0.42

